In [1]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from delta.tables import *
from pyspark.sql.functions import *
from pyspark.sql.functions import sum as _sum

In [2]:
conf = SparkConf()
conf.set("spark.hadoop.fs.s3a.impl",
                 "org.apache.hadoop.fs.s3a.S3AFileSystem")
conf.set("spark.hadoop.fs.s3a.access.key", "admin")
conf.set("spark.hadoop.fs.s3a.secret.key", "123456789")
conf.set("spark.hadoop.fs.s3a.endpoint", "http://localhost:9000")
conf.set("spark.hadoop.fs.s3a.path.style.access", "true")
conf.set("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")
conf.set('spark.hadoop.fs.s3a.aws.credentials.provider',
            'org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider')
conf.set(
    "spark.jars.packages",
    "io.delta:delta-core_2.12:1.0.0,"
    "org.apache.hadoop:hadoop-aws:3.1.1,"
    "com.amazonaws:aws-java-sdk:1.11.271,"
    "com.amazonaws:aws-java-sdk-bundle:1.11.271,"
    "software.amazon.awssdk:url-connection-client:2.15.40",
)
conf.set("spark.sql.extensions",
            "io.delta.sql.DeltaSparkSessionExtension")
conf.set(
    "spark.sql.catalog.spark_catalog",
    "org.apache.spark.sql.delta.catalog.DeltaCatalog",
)
conf.set("spark.databricks.delta.merge.repartitionBeforeWrite.enabled","true")

spark = (
    SparkSession
    .builder
    .config(conf=conf)
    .master("local[*]")
    .getOrCreate()
)

23/03/31 15:26:46 WARN Utils: Your hostname, ducdn-G3-3579 resolves to a loopback address: 127.0.1.1; using 192.168.1.24 instead (on interface wlo1)
23/03/31 15:26:46 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/ducdn/Documents/spark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ducdn/.ivy2/cache
The jars for the packages stored in: /home/ducdn/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
com.amazonaws#aws-java-sdk added as a dependency
com.amazonaws#aws-java-sdk-bundle added as a dependency
software.amazon.awssdk#url-connection-client added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-db2d89dd-27bb-441a-983e-373afcf551f3;1.0
	confs: [default]
	found io.delta#delta-core_2.12;1.0.0 in central
	found org.antlr#antlr4;4.7 in central
	found org.antlr#antlr4-runtime;4.7 in central
	found org.antlr#antlr-runtime;3.5.2 in central
	found org.antlr#ST4;4.0.8 in central
	found org.abego.treelayout#org.abego.treelayout.core;1.0.3 in central
	found org.glassfish#javax.json;1.0.4 in central
	found com.ibm.icu#icu4j;58.2 in central
	found org.apache.hadoop#hadoop-aws;3.1.1 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.271 in central

### Bronzen

In [22]:
orderDF = spark.read.format("delta").load("s3a://datalake/brozen/cdc.inventory.orders")
orderDF.show()

+---+-------------+------------+----------+---------+--------+----------+-------------------+
| op|        ts_ms|order_number|order_date|purchaser|quantity|product_id|before_order_number|
+---+-------------+------------+----------+---------+--------+----------+-------------------+
|  c|1680252250803|       10006|     19037|     1002|      29|       101|              10006|
|  u|1680251512287|       10004|     16852|     1003|      20|       107|              10004|
|  c|1680252977924|       10009|     19037|     1003|      34|       103|              10009|
|  r|1680248611130|       10002|     16817|     1002|       2|       105|              10002|
|  r|1680248611130|       10001|     16816|     1001|       1|       102|              10001|
|  c|1680252978931|       10010|     19037|     1002|      34|       105|              10010|
|  r|1680248611131|       10003|     16850|     1002|       2|       106|              10003|
|  c|1680252252815|       10008|     19037|     1002|      1

In [23]:
products = spark.read.format("delta").load("s3a://datalake/brozen/cdc.inventory.products")
products.show()

+---+-------------+---+------------------+--------------------+------+---------+
| op|        ts_ms| id|              name|         description|weight|before_id|
+---+-------------+---+------------------+--------------------+------+---------+
|  r|1680248611138|103|12-pack drill bits|12-pack of drill ...|   0.8|      103|
|  r|1680248611139|108|            jacket|water resistent b...|   0.1|      108|
|  u|1680252862220|105|            hammer|14oz carpenter's ...|  40.0|      105|
|  r|1680248611138|106|            hammer|16oz carpenter's ...|   1.0|      106|
|  r|1680248611138|104|            hammer|12oz carpenter's ...|  0.75|      104|
|  r|1680248611137|101|           scooter|Small 2-wheel sco...|  3.14|      101|
|  u|1680252639028|109|        spare tire|  24 inch spare tire|  50.0|      109|
|  r|1680248611137|102|       car battery|     12V car battery|   8.1|      102|
|  r|1680248611139|107|             rocks|box of assorted r...|   5.3|      107|
+---+-------------+---+-----

### Sliver

In [24]:
orderDF = spark.read.format("delta").load("s3a://datalake/demo")
orderDF.show(truncate=False)

+------------+----------+---------------------+---------+------------------+--------+----------+-------------+
|order_number|order_time|email                |purchaser|product_name      |quantity|unit_price|timestamp    |
+------------+----------+---------------------+---------+------------------+--------+----------+-------------+
|10001       |16816     |sally.thomas@acme.com|1001     |car battery       |1       |8.1       |1680248611130|
|10009       |19037     |ed@walker.com        |1003     |12-pack drill bits|34      |0.8       |1680252977924|
|10006       |19037     |gbailey@foobar.com   |1002     |scooter           |29      |3.14      |1680252250803|
|10003       |16850     |gbailey@foobar.com   |1002     |hammer            |2       |1.0       |1680248611131|
|10008       |19037     |gbailey@foobar.com   |1002     |hammer            |16      |1.0       |1680252252815|
|10002       |16817     |gbailey@foobar.com   |1002     |hammer            |2       |0.875     |1680248611130|
|

In [14]:
orderDF = spark.read.format("delta").load("s3a://datalake/gold/agg")
orderDF.show(truncate=False)

+------------------+-------+
|email             |quantiy|
+------------------+-------+
|gbailey@foobar.com|45     |
|ed@walker.com     |35     |
|ed@walker.com     |45     |
+------------------+-------+



In [ ]:
path_table = "s3a://datalake/brozen/cdc.inventory.orders"
df = spark.read.format("delta").option("versionAsOf", 0).load(path_table)
df.show()

In [ ]:
delta_table = DeltaTable.forPath(spark, "s3a://datalake/brozen/cdc.inventory.orders")
history = delta_table.history()
history.show()

In [ ]:
df = spark.read.format("delta").load("s3a://datalake/sliver")
df.show()